## 提取股价表

In [6]:
import pandas as pd

companyForm_path="../datasets/companyForm10Q/sec-edgar-filings/"
fundamentals_path="../datasets/fundamentals"
meta_path="../datasets/meta"
news_path="../datasets/news"

# 读取 AAPL 股票数据
aapl = pd.read_csv(f"{meta_path}/2024-08-01/AAPL_MSFT_GOOGL_processors.csv", parse_dates=['date'])

aapl = aapl[aapl['tic'] == 'AAPL']

# 查看前几行
aapl.head()



,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2024-08-01,AAPL,223.325586,223.435074,216.009808,217.343567,62501000.0,3.0,0.000000,220.201510,215.978636,100.000000,66.666667,100.000000,217.343567,217.343567,18.590000,0.0
3,2024-08-02,AAPL,218.129877,224.549865,216.696592,218.836578,105568600.0,4.0,0.033497,220.201510,215.978636,100.000000,66.666667,100.000000,218.090073,218.090073,23.389999,0.0
6,2024-08-05,AAPL,198.163247,212.506173,195.087635,208.295868,119548600.0,0.0,-0.284185,226.232836,203.417840,12.043140,-100.000000,90.499843,214.825338,214.825338,38.570000,0.0
9,2024-08-06,AAPL,204.344342,209.012512,200.134036,206.265350,69660500.0,1.0,-0.495019,225.335534,200.035148,10.247048,-67.411129,90.499843,212.685341,212.685341,27.709999,0.0
12,2024-08-07,AAPL,205.936902,212.645533,205.429281,208.843323,63516400.0,2.0,-0.496579,223.398637,200.435238,24.948013,-28.824957,62.629378,211.916937,211.916937,27.850000,0.0


## 提取新闻进入股价表

In [7]:
# 加载新闻数据
news_df = pd.read_csv(f"{news_path}/2024/AAPL_US_2024-12-31_news.csv", parse_dates=['date'])

# 只保留 AAPL 相关新闻

# 按日期聚合（将多个新闻拼接成一个字符串）
# 按日期将多列拼成一条字符串，再聚合
def merge_news(row):
    return f"Title:{row['title']}\n| Content: {row['content']}\n| Symbols: {row['symbols']}\n| Tags: {row['tags']}\n| Sentiment: {row['sentiment']}\n"

news_df['date'] = pd.to_datetime(news_df['date']).dt.date
news_df['date'] = pd.to_datetime(news_df['date'])

# 创建新列 news_text
news_df['news_text'] = news_df.apply(merge_news, axis=1)

# 按日期聚合（把当天所有新闻拼成一个大字符串）
news_grouped = news_df.groupby('date')['news_text'].apply(lambda x: ' '.join(x)).reset_index()

# 改列名
news_grouped.columns = ['date', 'news']

print(news_grouped.head())


import os
os.makedirs("../datasets/processed", exist_ok=True)

news_grouped.to_csv("../datasets/processed/aapl_with_news.csv", index=False)



        date                                               news
0 2024-08-01  Title:Dow Jones Futures Fall As Apple, Amazon ...
1 2024-08-02  Title:Apple stock finishes flat following Q3 e...
2 2024-08-03  Title:Warning: Warren Buffet is selling stocks...
3 2024-08-04  Title:Berkshire Hathaway's Massive $277M Cash ...
4 2024-08-05  Title:Apple’s Shift to AI Is Poised to Soften ...


In [8]:
# 将新闻合并到股价表中
aapl_with_news = pd.merge(aapl, news_grouped, on='date', how='left')

# 查看结果
aapl_with_news[['date', 'close', 'news']].head()

,date,close,news
0,2024-08-01,217.343567,"Title:Dow Jones Futures Fall As Apple, Amazon ..."
1,2024-08-02,218.836578,Title:Apple stock finishes flat following Q3 e...
2,2024-08-05,208.295868,Title:Apple’s Shift to AI Is Poised to Soften ...
3,2024-08-06,206.265350,Title:Google’s unlawful grip on search could s...
4,2024-08-07,208.843323,Title:3 Stocks to Dump as Investors Rotate to ...


## 情绪提取

In [13]:
import pandas as pd

# === 读取新闻数据 ===
news_df = pd.read_csv(f"{news_path}/2024/AAPL_US_2024-12-31_news.csv", parse_dates=['date'])

# === 过滤时间：只保留 09:00 之前的新闻 ===
news_df['date'] = pd.to_datetime(news_df['date'])  # 保证是 datetime 类型
# news_df = news_df[news_df['date'].dt.time < pd.to_datetime('09:00:00').time()]

# === 衍生出纯日期列，作为合并键 ===
news_df['date_only'] = news_df['date'].dt.date
news_df['date_only'] = pd.to_datetime(news_df['date_only'])  # 再转回 datetime 类型

# === 合并 title 和 sentiment（可根据需要调整）===
news_df['news_text'] = news_df['title'].fillna('') + " " + news_df['content'].fillna('')
news_df['sentiment'] = news_df['sentiment'].fillna('')

# === 按日期聚合新闻 ===
news_grouped = news_df.groupby('date_only').agg({
    'news_text': lambda x: ' '.join(x),
    'sentiment': lambda x: ','.join(x)
}).reset_index().rename(columns={"date_only": "date"})

# === 读取股价表，并合并新闻 ===
aapl = pd.read_csv(f"{meta_path}/2024-08-01/AAPL_MSFT_GOOGL_processors.csv", parse_dates=['date'])
aapl = aapl[aapl['tic'] == 'AAPL']
aapl_with_news = pd.merge(aapl, news_grouped, on='date', how='left')

# === 保存合并后的表 ===
aapl_with_news.to_csv("../datasets/processed/aapl_with_news_sentiment.csv", index=False)

print(aapl_with_news.head())

        date   tic        open        high         low       close  \
0 2024-08-01  AAPL  223.325586  223.435074  216.009808  217.343567   
1 2024-08-02  AAPL  218.129877  224.549865  216.696592  218.836578   
2 2024-08-05  AAPL  198.163247  212.506173  195.087635  208.295868   
3 2024-08-06  AAPL  204.344342  209.012512  200.134036  206.265350   
4 2024-08-07  AAPL  205.936902  212.645533  205.429281  208.843323   

        volume  day      macd     boll_ub     boll_lb      rsi_30      cci_30  \
0   62501000.0  3.0  0.000000  220.201510  215.978636  100.000000   66.666667   
1  105568600.0  4.0  0.033497  220.201510  215.978636  100.000000   66.666667   
2  119548600.0  0.0 -0.284185  226.232836  203.417840   12.043140 -100.000000   
3   69660500.0  1.0 -0.495019  225.335534  200.035148   10.247048  -67.411129   
4   63516400.0  2.0 -0.496579  223.398637  200.435238   24.948013  -28.824957   

        dx_30  close_30_sma  close_60_sma        vix  turbulence  \
0  100.000000    217.343

## 提取财报，合并到news_aapl中

In [ ]:
import os
import pandas as pd
import re

def extract_report_date_from_txt(txt_path):
    try:
        with open(txt_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
            # 提取 endDate
            match = re.search(r'<endDate>(\d{4}-\d{2}-\d{2})</endDate>', text)
            if match:
                return pd.to_datetime(match.group(1))
    except Exception as e:
        print(f"读取 {txt_path} 出错: {e}")
    return pd.NaT

def load_financials(base_path):
    all_data = []
    for form_type in ['10-K', '10-Q','8-K']:
        form_path = os.path.join(base_path, 'AAPL', form_type)
        if not os.path.exists(form_path):
            print(f"路径不存在: {form_path}")
            continue

        for acc_folder in os.listdir(form_path):
            folder_path = os.path.join(form_path, acc_folder)
            csv_files = [f for f in os.listdir(folder_path) if f.endswith('_Fields.csv')]
            txt_file = os.path.join(folder_path, 'full-submission.txt')

            # 用新的日期提取函数
            report_date = extract_report_date_from_txt(txt_file) if os.path.exists(txt_file) else pd.NaT

            for csv_file in csv_files:
                full_path = os.path.join(folder_path, csv_file)
                try:
                    df = pd.read_csv(full_path)
                    df['form_type'] = form_type
                    df['report_date'] = report_date
                    all_data.append(df)
                except Exception as e:
                    print(f"读取 {full_path} 出错: {e}")

    return pd.concat(all_data, ignore_index=True)

# 使用函数读取数据
financial_df = load_financials(companyForm_path)
print(financial_df.head())
financial_df.to_csv("../datasets/processed/financial_report_pre.csv", index=False)


# 保留你关心的字段
fields = [
    "Total Revenue", "Net Income", "EBITDA", "Operating Income", "R&D", "SG&A", 
    "Total Assets", "Total Liabilities", "Long Term Debt", "Current Liabilities", 
    "Cash and Equivalents", "Stockholders Equity", "Operating Cash Flow", 
    "Free Cash Flow", "Capital Expenditure", "Net Profit Margin", 
    "Operating Margin", "R&D Intensity", "SG&A Ratio", "ROA", "ROE", 
    "Debt to Equity", "Long-term Debt Ratio", "Current Ratio", "FCF Margin"
]

# import pandas as pd

# # 假设financial_df是你读入的DataFrame
# # 透视表，把name做列，value做值
# pivot_df = financial_df.pivot_table(index=['report_date', 'form_type'], 
#                                     columns='name', 
#                                     values='value', 
#                                     aggfunc='first').reset_index()

# print(pivot_df.head())
# pivot_df.to_csv("../datasets/processed/aapl_with_news3.csv", index=False)

import pandas as pd
import json

# 读取原始数据
df = pd.read_csv("../datasets/processed/financial_report_pre.csv")  # 或你已有的 dataframe

# 仅保留需要列
df = df[["report_date", "form_type", "name", "contextRef", "unitRef", "value"]]

# 分组后把每组的所有信息整理成 dict 列表
def combine_rows(sub_df):
    return sub_df.apply(lambda row: {
        "name": row["name"],
        "value": row["value"],
        "contextRef": row["contextRef"],
        "unitRef": row["unitRef"]
    }, axis=1).tolist()

grouped_df = df.groupby(["report_date", "form_type"]).apply(combine_rows).reset_index()
grouped_df.columns = ["report_date", "form_type", "all_fields"]

# 可选：如果你想把列表转成 JSON 字符串形式
grouped_df["all_fields"] = grouped_df["all_fields"].apply(json.dumps)

# 保存或使用
grouped_df.to_csv("../datasets/processed/financial_report.csv", index=False)
# print(grouped_df.head())

grouped_df = grouped_df.sort_values("report_date")
print(grouped_df.head(20))


                                          name  \
0                            dei:AmendmentFlag   
1                  dei:DocumentFiscalYearFocus   
2                dei:DocumentFiscalPeriodFocus   
3                    dei:EntityCentralIndexKey   
4  us-gaap:PropertyPlantAndEquipmentUsefulLife   

                                          contextRef unitRef       value  \
0  i4a4fb50fc2db4070adb20304175890ef_D20210926-20...     NaN       false   
1  i4a4fb50fc2db4070adb20304175890ef_D20210926-20...     NaN        2022   
2  i4a4fb50fc2db4070adb20304175890ef_D20210926-20...     NaN          FY   
3  i4a4fb50fc2db4070adb20304175890ef_D20210926-20...     NaN  0000320193   
4  i29f7971951944e36829aff20a44149a4_D20210926-20...     NaN         P1Y   

  form_type report_date  
0      10-K  2022-09-24  
1      10-K  2022-09-24  
2      10-K  2022-09-24  
3      10-K  2022-09-24  
4      10-K  2022-09-24  
   report_date form_type                                         all_fields
0   2019-0

/tmp/ipykernel_135829/3422420633.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby(["report_date", "form_type"]).apply(combine_rows).reset_index()


## 最后的合并

In [ ]:
# # 确保两个表的时间格式正确
# aapl_with_news["date"] = pd.to_datetime(aapl_with_news["date"])
# grouped_df["report_date"] = pd.to_datetime(grouped_df["report_date"])

# # 都要排序！
# aapl_with_news = aapl_with_news.sort_values("date")
# grouped_df = grouped_df.sort_values("report_date")
# print(grouped_df.head(20))

# # 用 merge_asof 对齐：date 对应 <= 最近的 report_date
# merged_df = pd.merge_asof(
#     aapl_with_news,
#     grouped_df,
#     left_on="date",
#     right_on="report_date",
#     direction="backward"  # 意思是 “找之前最近的一个报告”
# )

# # （可选）去掉 report_date 列，避免重复
# merged_df = merged_df.drop(columns=["report_date"])
# merged_df.to_csv("../datasets/processed/financial_final.csv", index=False)

# print(merged_df.head(20))

import pandas as pd

# 时间格式统一
aapl_with_news["date"] = pd.to_datetime(aapl_with_news["date"])
grouped_df["report_date"] = pd.to_datetime(grouped_df["report_date"])

# 精确按日期合并，只匹配同一天有财报发布的新闻
merged_df = pd.merge(
    aapl_with_news,
    grouped_df,
    left_on="date",
    right_on="report_date",
    how="left"  # left join 保留所有新闻，report_date 匹配不到就 NaN
)

# 去掉 report_date 列
merged_df = merged_df.drop(columns=["report_date"])

# 保存结果
merged_df.to_csv("../datasets/processed/financial_final.csv", index=False)

# 看看匹配效果
print(merged_df.head(20))
print(merged_df[merged_df["form_type"].notna()].head(20))  # 看看有哪些匹配到了财报
print(merged_df[merged_df["date"] == "2024-08-01"])  # 检查 2024-08-01 是不是没有匹配

print(merged_df[merged_df["date"] == "2024-08-01"]["sentiment"])  # 检查 2024-08-01 是不是没有匹配
obj=merged_df[merged_df["date"] == "2024-08-01"]["sentiment"]

for i in obj:
    print(i)


         date   tic        open        high         low       close  \
0  2024-08-01  AAPL  223.325586  223.435074  216.009808  217.343567   
1  2024-08-02  AAPL  218.129877  224.549865  216.696592  218.836578   
2  2024-08-05  AAPL  198.163247  212.506173  195.087635  208.295868   
3  2024-08-06  AAPL  204.344342  209.012512  200.134036  206.265350   
4  2024-08-07  AAPL  205.936902  212.645533  205.429281  208.843323   
5  2024-08-08  AAPL  212.117996  213.202919  207.857921  212.317062   
6  2024-08-09  AAPL  211.112701  215.770909  210.983301  215.233429   
7  2024-08-12  AAPL  215.313158  218.741096  214.844804  216.768036   
8  2024-08-13  AAPL  218.242841  221.112757  218.242841  220.494934   
9  2024-08-14  AAPL  219.797394  222.248768  218.930431  220.943359   
10 2024-08-15  AAPL  223.813271  224.560644  221.979705  223.932846   
11 2024-08-16  AAPL  223.135650  226.035461  222.866592  225.258194   
12 2024-08-19  AAPL  224.929337  225.198395  222.258717  225.098740   
13 202